In [1]:
pwd

'/home/js7561/dev/milsed/notebooks'

In [2]:
cd ../models/

/home/js7561/dev/milsed/models


In [3]:
pwd

'/home/js7561/dev/milsed/models'

In [4]:
ls

00_setup.py*    02_train.py*  __pycache__/
01_prepare.py*  __init__.py   resources/


In [6]:
import os

In [7]:
os.system('python 02_train.py /beegfs/js7561/datasets/dcase2017/task4_official/train/features_silence/')

256

In [32]:
import argparse
import os
import sys
from glob import glob
import six
import pickle
import json

import pandas as pd
import keras as K

from sklearn.model_selection import ShuffleSplit

import pescador
import librosa
import milsed.utils
from jams.util import smkdirs

OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources'

In [33]:
pescador.__version__

'1.0.0'

In [63]:
def make_sampler(max_samples, duration, pump, seed):

    n_frames = librosa.time_to_frames(duration,
                                      sr=pump['mel'].sr,
                                      hop_length=pump['mel'].hop_length)[0]

    return pump.sampler(max_samples, n_frames-1, random_state=seed)


def data_sampler(fname, sampler):
    '''Generate samples from a specified h5 file'''
    for datum in sampler(milsed.utils.load_h5(fname)):
        yield datum


def data_generator(working, tracks, sampler, k, augment=True, batch_size=32,
                   **kwargs):
    '''Generate a data stream from a collection of tracks and a sampler'''

    seeds = []

    for track in tracks:
        fname = os.path.join(working,
                             os.path.extsep.join([str(track), 'h5']))
        seeds.append(pescador.Streamer(data_sampler, fname, sampler))

        if augment:
            for fname in sorted(glob(os.path.join(working,
                                                  '{}.*.h5'.format(track)))):
                seeds.append(pescador.Streamer(data_sampler, fname, sampler))

    # Send it all to a mux
    mux = pescador.Mux(seeds, k, **kwargs)

    if batch_size == 1:
        return mux
    else:
        return pescador.BufferedStreamer(mux, batch_size)


def keras_tuples(gen, inputs=None, outputs=None):

    if isinstance(inputs, six.string_types):
        if isinstance(outputs, six.string_types):
            # One input, one output
            for datum in gen:
                yield (datum[inputs], datum[outputs])
        else:
            # One input, multi outputs
            for datum in gen:
                yield (datum[inputs], [datum[o] for o in outputs])
    else:
        if isinstance(outputs, six.string_types):
            for datum in gen:
                yield ([datum[i] for i in inputs], datum[outputs])
        else:
            # One input, multi outputs
            for datum in gen:
                yield ([datum[i] for i in inputs],
                       [datum[o] for o in outputs])


def construct_model(pump, alpha):

    model_inputs = ['mel/mag']

    # Build the input layer
    layers = pump.layers()

    x_mag = layers['mel/mag']

    # Apply batch normalization
    x_bn = K.layers.BatchNormalization()(x_mag)

    x_sq = milsed.layers.SqueezeLayer()(x_bn)

    # First convolutional filter: a single 3-frame filters
    conv1 = K.layers.Convolution1D(64, 3,
                                   padding='same',
                                   activation='relu',
                                   kernel_initializer='he_uniform')(x_sq)
                                   # data_format='channels_last')(x_sq)

    # First recurrent layer: a 128-dim bidirectional gru
    rnn1 = K.layers.Bidirectional(K.layers.GRU(128,
                                               return_sequences=True))(conv1)

    n_classes = pump.fields['static/tags'].shape[0]

    p0 = K.layers.Dense(n_classes, activation='sigmoid')

    p_dynamic = K.layers.TimeDistributed(p0, name='dynamic/tags')(rnn1)

    p_static = milsed.layers.SoftMaxPool(alpha=alpha,
                                         axis=1,
                                         name='static/tags')(p_dynamic)

    model = K.models.Model([x_mag],
                           [p_dynamic, p_static])

    model_outputs = ['dynamic/tags', 'static/tags']

    return model, model_inputs, model_outputs

In [85]:
def train(working, alpha, max_samples, duration, rate,
          batch_size, epochs, epoch_size, validation_size,
          early_stopping, reduce_lr, seed, version):
    '''
    Parameters
    ----------
    working : str
        directory that contains the experiment data (h5)

    alpha : float > 0
        Alpha parameter for softmax

    max_samples : int
        Maximum number of samples per streamer

    duration : float
        Duration of training patches

    batch_size : int
        Size of batches

    rate : int
        Poisson rate for pescador

    epochs : int
        Maximum number of epoch

    epoch_size : int
        Number of batches per epoch

    validation_size : int
        Number of validation batches

    early_stopping : int
        Number of epochs before early stopping

    reduce_lr : int
        Number of epochs before reducing learning rate

    seed : int
        Random seed

    version: str
        Identifier for current model version (model ID)
    '''

    print('Load pump..')
    # Load the pump
    with open(os.path.join(OUTPUT_PATH, 'pump.pkl'), 'rb') as fd:
        pump = pickle.load(fd)

    # Build the sampler
    sampler = make_sampler(max_samples, duration, pump, seed)

    print('Build model..')
    # Build the model
    model, inputs, outputs = construct_model(pump, alpha)

    print('Load index..')
    # Load the training data
    idx_train_ = pd.read_json(os.path.join(OUTPUT_PATH, 'index_train.json'))

    print('Create splits..')
    # Split the training data into train and validation
    splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                               random_state=seed)
    train, val = next(splitter_tv.split(idx_train_))

    idx_train = idx_train_.iloc[train]
    idx_val = idx_train_.iloc[val]

    print('Create train generator..')
    gen_train = data_generator(working,
                               idx_train['id'].values, sampler, epoch_size,
                               augment=False,
                               lam=rate,
                               batch_size=batch_size,
                               revive=True,
                               random_state=seed)

    print('Keras tuples..')
    gen_train = keras_tuples(gen_train(), inputs=inputs, outputs=['static/tags'])

    print('Create validation generator..')
    gen_val = data_generator(working,
                             idx_val['id'].values, sampler, len(idx_val),
                             augment=False,
                             batch_size=batch_size,
                             revive=True,
                             random_state=seed)
    print('Keras tuples..')
    gen_val = keras_tuples(gen_val(), inputs=inputs, outputs=['static/tags'])

    print('Compile model..')
    loss = {'static/tags': 'binary_crossentropy'}
    metrics = {'static/tags': 'accuracy'}
    monitor = 'val_loss'

    model.compile(K.optimizers.Adam(), loss=loss, metrics=metrics)

    # Store the model
    model_spec = K.utils.serialize_keras_object(model)
    with open(os.path.join(OUTPUT_PATH, version, 'model_spec.pkl'), 'wb') as fd:
        pickle.dump(model_spec, fd)

    # Construct the weight path
    weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')

    # Build the callbacks
    cb = []
    cb.append(K.callbacks.ModelCheckpoint(weight_path,
                                          save_best_only=True,
                                          verbose=1,
                                          monitor=monitor))

    cb.append(K.callbacks.ReduceLROnPlateau(patience=reduce_lr,
                                            verbose=1,
                                            monitor=monitor))

    cb.append(K.callbacks.EarlyStopping(patience=early_stopping,
                                        verbose=1,
                                        monitor=monitor))

    print('Fit..')
    # Fit the model
    model.fit_generator(gen_train, epoch_size, epochs,
                        validation_data=gen_val,
                        validation_steps=validation_size,
                        callbacks=cb)


In [86]:
working = '/beegfs/js7561/datasets/dcase2017/task4_official/train/features_silence/'

In [87]:
version = milsed.utils.increment_version(os.path.join(OUTPUT_PATH,
                                                          'version.txt'))
print('Model version: {}'.format(version))
smkdirs(os.path.join(OUTPUT_PATH, version))

Model version: 4e29016.7


In [ ]:
train(working, 1.0, 128, 10.0, 8, 32, 100, 512, 1024, 20, 10, 20170612, version)

Load pump..
Build model..
Load index..
Create splits..
Create train generator..
Keras tuples..
Create validation generator..
Keras tuples..
Compile model..
Fit..


/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/ipykernel_launcher.py:99: UserWarning: Output "dynamic/tags" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dynamic/tags" during training.


Epoch 1/100
511/512 [============================>.] - ETA: 0s - loss: 0.1679 - static/tags_loss: 0.1679 - static/tags_acc: 0.9404

### Step by step

In [42]:
params = {'working': '/beegfs/js7561/datasets/dcase2017/task4_official/train/features_silence/',
          'alpha':1.0, 
          'max_samples': 128, 
          'duration': 10.0, 
          'rate': 8,
          'batch_size': 32, 
          'epochs': 100, 
          'epoch_size': 512, 
          'validation_size': 1024,
          'early_stopping': 20, 
          'reduce_lr': 10, 
          'seed': 20170612, 
          'version': version}

In [80]:
working = '/beegfs/js7561/datasets/dcase2017/task4_official/train/features_silence/'
alpha =1.0
max_samples = 128
duration = 10.0
rate = 8
batch_size = 32
epochs = 100
epoch_size = 512
validation_size = 1024
early_stopping = 20
reduce_lr = 10
seed = 20170612
version = version

In [81]:
print('Load pump..')
# Load the pump
with open(os.path.join(OUTPUT_PATH, 'pump.pkl'), 'rb') as fd:
    pump = pickle.load(fd)

# Build the sampler
sampler = make_sampler(max_samples, duration, pump, seed)

print('Build model..')
# Build the model
model, inputs, outputs = construct_model(pump, alpha)

print('Load index..')
# Load the training data
idx_train_ = pd.read_json(os.path.join(OUTPUT_PATH, 'index_train.json'))

print('Create splits..')
# Split the training data into train and validation
splitter_tv = ShuffleSplit(n_splits=1, test_size=0.25,
                           random_state=seed)
train, val = next(splitter_tv.split(idx_train_))

idx_train = idx_train_.iloc[train]
idx_val = idx_train_.iloc[val]

print('Create train generator..')
gen_train = data_generator(working,
                           idx_train['id'].values, sampler, epoch_size,
                           augment=False,
                           lam=rate,
                           batch_size=batch_size,
                           revive=True,
                           random_state=seed)

print('Keras tuples..')
gen_train = keras_tuples(gen_train(), inputs=inputs, outputs=outputs)

print('Create validation generator..')
gen_val = data_generator(working,
                         idx_val['id'].values, sampler, len(idx_val),
                         augment=False,
                         batch_size=batch_size,
                         revive=True,
                         random_state=seed)
print('Keras tuples..')
gen_val = keras_tuples(gen_val(), inputs=inputs, outputs=outputs)

Load pump..
Build model..
Load index..
Create splits..
Create train generator..
Keras tuples..
Create validation generator..
Keras tuples..


In [84]:
outputs

['dynamic/tags', 'static/tags']

In [82]:
print('Compile model..')
loss = {'static/tags': 'binary_crossentropy'}
metrics = {'static/tags': 'accuracy'}
monitor = 'val_loss'

model.compile(K.optimizers.Adam(), loss=loss, metrics=metrics)

# Store the model
model_spec = K.utils.serialize_keras_object(model)
with open(os.path.join(OUTPUT_PATH, version, 'model_spec.pkl'), 'wb') as fd:
    pickle.dump(model_spec, fd)

# Construct the weight path
weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')

# Build the callbacks
cb = []
cb.append(K.callbacks.ModelCheckpoint(weight_path,
                                      save_best_only=True,
                                      verbose=1,
                                      monitor=monitor))

cb.append(K.callbacks.ReduceLROnPlateau(patience=reduce_lr,
                                        verbose=1,
                                        monitor=monitor))

cb.append(K.callbacks.EarlyStopping(patience=early_stopping,
                                    verbose=1,
                                    monitor=monitor))


Compile model..


/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Output "dynamic/tags" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dynamic/tags" during training.
  


In [83]:
for data in gen_train:
    break

KeyError: 'dynamic/tags'

In [72]:
data = [x for x in gen_val]

KeyError: 'dynamic/tags'

In [54]:
data

NameError: name 'data' is not defined

In [55]:
import librosa

In [59]:
librosa.time_to_frames(10.0, sr=44100, hop_length=1024)

array([430])

In [60]:
import numpy as np

In [62]:
np.random.randint(0, 1)

0

In [73]:
pwd

'/home/js7561/dev/milsed/models'

In [74]:
x = milsed.utils.load_h5('/beegfs/js7561/datasets/dcase2017/task4_official/test/features_silence/Y---lTs1dxhU_30.000_40.000.h5')

In [75]:
x

{'mel/mag': array([[[[-22.96606445],
          [-20.01185226],
          [-22.13051414],
          ..., 
          [-79.51634216],
          [-79.71124268],
          [-79.71455383]],
 
         [[-17.16335297],
          [-21.88430023],
          [-23.585989  ],
          ..., 
          [-80.        ],
          [-80.        ],
          [-80.        ]],
 
         [[-22.02036667],
          [-26.22042274],
          [-28.48296738],
          ..., 
          [-80.        ],
          [-80.        ],
          [-80.        ]],
 
         ..., 
         [[-34.1590538 ],
          [-35.00361633],
          [-31.60760498],
          ..., 
          [-80.        ],
          [-80.        ],
          [-80.        ]],
 
         [[-34.37856293],
          [-36.13519669],
          [-29.77634811],
          ..., 
          [-80.        ],
          [-80.        ],
          [-80.        ]],
 
         [[-37.751194  ],
          [-44.70385361],
          [-41.91004944],
          ..., 
     